# 08_Logistic_Regression

1. Design model (input, output size, forward pass)
2. Construct loss and optimization
3. Training the loop
    - forward pass: compute prediction and loss
    - backward pass: compute gradient
    - update weights

In [26]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Data Preparation

In [27]:
# 0 Data Preparation
bc = datasets.load_breast_cancer()
X = bc.data
y = bc.target
print(type(X), type(y))
print(X.shape, y.shape)

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(569, 30) (569,)


In [28]:
# scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

# 2. Neural Network

In [29]:
# model
# f = w*x+b, sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

In [31]:
# loss and optimizer
loss = nn.BCELoss()

learning_rate = 0.01
n_iters = 100
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [34]:
# Training Loop
for epoch in range(n_iters):
    # Prediction
    y_pred = model(X_train)
    
    # loss
    l = loss(y_pred, y_train)
    
    # gradient
    l.backward()
    
    # Update weights
    optimizer.step()
    
    # zero w.grad
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        w, b = model.parameters()
        print("Epoch {0}: loss={1}".format(epoch+1, l.item()))

Epoch 10: loss=0.3905348479747772
Epoch 20: loss=0.35248032212257385
Epoch 30: loss=0.3235328495502472
Epoch 40: loss=0.30063799023628235
Epoch 50: loss=0.281982421875
Epoch 60: loss=0.26642072200775146
Epoch 70: loss=0.2531931400299072
Epoch 80: loss=0.2417750060558319
Epoch 90: loss=0.23179198801517487
Epoch 100: loss=0.22296933829784393


# 3. Evaluate Performance

In [38]:
# If we don't use this statement, this will be part of computation graph and tracked gradient calculation for us 
# We don't want this, becaues we are done
with torch.no_grad():
    y_predicted  = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print("accurancy = {0}".format(acc))

accurancy = 0.9210526347160339
